In [2]:
from faker import Faker
import mysql.connector
import re
import unicodedata
from datetime import datetime
import random

# Połączenie z bazą danych
con = mysql.connector.connect(
    host="giniewicz.it",
    user="team19",
    password="te@mzaiq",
    database="team19"
)

mycursor = con.cursor()

# Inicjalizacja obiektu Faker
fake = Faker('pl_PL')  # Polski lokalizator

# Funkcja do usuwania polskich znaków diakrytycznych z tekstu
def remove_polish_chars(text):
    return ''.join(
        c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn'
    )

# Lista popularnych domen e-mail
domeny = ["gmail.com", "yahoo.com", "outlook.com", "wp.pl", "interia.pl", "onet.pl"]

# Funkcja do generowania e-maila na podstawie imienia i nazwiska, usuwanie polskich znaków
def generuj_email(imie, nazwisko):
    domena = random.choice(domeny)  # Wybieramy losową domenę z listy
    return f"{remove_polish_chars(imie.lower())}.{remove_polish_chars(nazwisko.lower())}@{domena}"

# Generowanie i wstawianie nowych danych do tabeli
for _ in range(300):
    Imie = fake.first_name()
    Nazwisko = fake.last_name()
    
    # Generowanie e-maila na podstawie imienia i nazwiska, usuwanie polskich znaków
    Email = generuj_email(Imie, Nazwisko)
    
    # Generowanie numeru telefonu z prefiksem +48
    Numer_telefonu = "+48 " + " ".join([str(int(re.sub(r'\D', '', fake.phone_number())))[-9:].rjust(9, '0')[i:i+3] for i in range(0, 9, 3)])
    
    Adres = fake.address().replace("\n", ", ")
    
    # Generowanie danych kontaktowych bliskich
    Imie_bliskiej_osoby = fake.first_name()

    # 80% szans, że nazwisko bliskiej osoby będzie takie samo jak klienta, 20% szans, że będzie inne
    if random.random() < 0.8:
        Nazwisko_bliskiej_osoby = Nazwisko
    else:
        Nazwisko_bliskiej_osoby = fake.last_name()

    Numer_telefonu_bliskiej_osoby = "+48 " + " ".join([str(int(re.sub(r'\D', '', fake.phone_number())))[-9:].rjust(9, '0')[i:i+3] for i in range(0, 9, 3)])

    # Tworzymy pełne dane kontaktowe bliskich
    Dane_kontaktowe_bliskich = f"{Imie_bliskiej_osoby} {Nazwisko_bliskiej_osoby}, {Numer_telefonu_bliskiej_osoby}"

    Data_urodzenia = fake.date_of_birth(minimum_age=18, maximum_age=80).strftime('%Y-%m-%d')

    # Wstawianie danych do tabeli Klienci
    mycursor.execute(""" 
        INSERT INTO Klienci (Imie, Nazwisko, Numer_telefonu, Email, Adres, Dane_kontaktowe_bliskich, Data_urodzenia)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (Imie, Nazwisko, Numer_telefonu, Email, Adres, Dane_kontaktowe_bliskich, Data_urodzenia))

# Zatwierdź zmiany i zamknij połączenie
con.commit()
print(f"Wstawiono {300} rekordów do tabeli Klienci.")

mycursor.close()
con.close()


Wstawiono 100 rekordów do tabeli Klienci.
